In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
import pandas as pd

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://section.blog.naver.com/")

driver.find_element(
    By.CSS_SELECTOR,
    "#header > div.header_common > div > div.area_search > form > fieldset > div > input",
).send_keys("뜨개카페")
driver.find_element(
    By.CSS_SELECTOR,
    "#header > div.header_common > div > div.area_search > form > fieldset > div > input",
).send_keys(Keys.ENTER)

# 페이지 반복 
cafe = []
for i in range(1,12):
    url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={i}&rangeType=ALL&orderBy=sim&keyword=%EB%9C%A8%EA%B0%9C%EC%B9%B4%ED%8E%98"
    driver.get(url)
    time.sleep(2)
    
    blog_links = []

    # 한페이지에 7개 블로그 글 
    blog_cards = driver.find_elements(
        By.CSS_SELECTOR, "#content > section > div.area_list_search > div"
    )[:7]

    # 블로그 주소, id, no 추출
    for card in blog_cards:
        try:
            elem = card.find_element(By.CSS_SELECTOR, "div > div.info_post > div.desc > a.desc_inner")
            blog_links.append(elem.get_attribute("href"))
        except:
            continue
    

    for blog_url in blog_links:
        try:
            blogID = blog_url.split("/")[-2]
            logNo = blog_url.split("/")[-1]
            # 블로그 접속
            driver.get(f"https://blog.naver.com/PostView.naver?blogId={blogID}&logNo={logNo}")
            time.sleep(2)

            try:
                # 지도 정보가 있는지 확인
                map_elem = driver.find_element(
                    By.CSS_SELECTOR,
                    "div.se-module.se-module-map-text > a"
                )
                map_data_raw = map_elem.get_attribute("data-linkdata")
                map_data = json.loads(map_data_raw)
                # 블로그 내용 크롤링
                blog = driver.find_element(By.CSS_SELECTOR, "#printPost1 > tbody > tr > td.bcc").text

                cafe.append({
                    "카페 이름": map_data["name"],
                    "카페 주소": map_data["address"],
                    "위도": map_data["latitude"],
                    "경도": map_data["longitude"],
                    "블로그내용" : blog
                })
                
            except:
                print("지도정보 없음")

        except Exception as e:
            print("오류 발생:", e)
            continue




In [ ]:
df_cafe = pd.DataFrame(cafe)
df_cafe